In [1]:
import os
os.chdir('/content/drive/MyDrive/ML and AI Projects/Ineuron_Practical')

In [2]:
pwd

'/content/drive/MyDrive/ML and AI Projects/Ineuron_Practical'

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import BatchNormalization

In [9]:
cifar100 = tf.keras.datasets.cifar100

In [10]:
(xtrain,ytrain),(xtest,ytest)= cifar100.load_data()
print('Shape of xtrain data is {}'.format(xtrain.shape))
print('Shape of ytrain data is {}'.format(ytrain.shape))
print('Shape of xtest data is {}'.format(xtest.shape))
print('Shape of ytest data is {}'.format(ytest.shape))

Shape of xtrain data is (50000, 32, 32, 3)
Shape of ytrain data is (50000, 1)
Shape of xtest data is (10000, 32, 32, 3)
Shape of ytest data is (10000, 1)


In [11]:
# standardize the training data and test data and also converting the ytrain and ytest into the sparse matrix
from keras.utils import np_utils
xtrain_scaled = preprocess_input(xtrain)
xtest_scaled  = preprocess_input(xtest)
num_classes   = 100
ytrain_sparse = np_utils.to_categorical(ytrain,num_classes)
ytest_sparse = np_utils.to_categorical(ytest,num_classes)

In [27]:
# importing the Resnet 50 Model
resnet50 = ResNet50(weights='imagenet',include_top=False,input_shape=(256,256,3)) 

In [28]:
resnet50.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [29]:
for layer in resnet50.layers:
  if isinstance(layer,BatchNormalization):
    layer.trainable = True
  else:
    layer.trainable = False

resnet50.summary()     
 
 

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import UpSampling2D,Dense,Flatten,Dropout,GlobalAveragePooling2D,BatchNormalization

model = Sequential()
model.add(UpSampling2D())
model.add(UpSampling2D())
model.add(UpSampling2D())
model.add(resnet50)
model.add(GlobalAveragePooling2D())
model.add(Dense(256,activation='relu'))
model.add(Dropout(.25))
model.add(BatchNormalization())
model.add(Dense(num_classes,activation='softmax'))


In [31]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [32]:
import time
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

modelchkpt = ModelCheckpoint('Resnet50_best_model_weight.h5',verbose=2,save_best_only=True)
                            
earlystopping = EarlyStopping(patience=20,verbose=1,restore_best_weights=True)

history = model.fit(xtrain_scaled, ytrain_sparse, steps_per_epoch=20, epochs=50, batch_size=64, validation_data=(xtest_scaled,ytest_sparse ),callbacks=[modelchkpt,earlystopping])

Epoch 1/50
20/20 [==============================] - 44s 1s/step - loss: 4.5952 - accuracy: 0.0531 - val_loss: 4.5613 - val_accuracy: 0.0595

Epoch 00001: val_loss improved from inf to 4.56130, saving model to Resnet50_best_model_weight.h5


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/50
20/20 [==============================] - 23s 1s/step - loss: 3.7048 - accuracy: 0.1656 - val_loss: 3.7921 - val_accuracy: 0.1500

Epoch 00002: val_loss improved from 4.56130 to 3.79213, saving model to Resnet50_best_model_weight.h5
Epoch 3/50
20/20 [==============================] - 23s 1s/step - loss: 3.2485 - accuracy: 0.2367 - val_loss: 3.3979 - val_accuracy: 0.2214

Epoch 00003: val_loss improved from 3.79213 to 3.39794, saving model to Resnet50_best_model_weight.h5
Epoch 4/50
20/20 [==============================] - 23s 1s/step - loss: 2.9778 - accuracy: 0.2875 - val_loss: 3.0572 - val_accuracy: 0.2700

Epoch 00004: val_loss improved from 3.39794 to 3.05717, saving model to Resnet50_best_model_weight.h5
Epoch 5/50
20/20 [==============================] - 23s 1s/step - loss: 2.7129 - accuracy: 0.3359 - val_loss: 2.7312 - val_accuracy: 0.3293

Epoch 00005: val_loss improved from 3.05717 to 2.73120, saving model to Resnet50_best_model_weight.h5
Epoch 6/50
20/20 [==========